In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
!pip install neattext


In [46]:
import pandas as pd
import neattext.functions as nfx
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
#!pip install neattext

In [48]:
data = pd.read_csv('/content/drive/MyDrive/Final Year/udemy_courses.csv')

In [49]:
data.columns

Index(['course_id', 'course_title', 'url', 'is_paid', 'price',
       'num_subscribers', 'num_reviews', 'num_lectures', 'level',
       'content_duration', 'published_timestamp', 'subject'],
      dtype='object')

In [50]:
data.head(1)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance


In [51]:
data.isnull().sum()

,0
course_id,0
course_title,0
url,0
is_paid,0
price,0
num_subscribers,0
num_reviews,0
num_lectures,0
level,0
content_duration,0


In [52]:
data.duplicated().any()

np.True_

In [53]:
data[data.duplicated()]

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
787,837322,Essentials of money value: Get a financial Life !,https://www.udemy.com/essentials-of-money-value/,True,20,0,0,20,All Levels,0.616667,2016-05-16T18:28:30Z,Business Finance
788,1157298,Introduction to Forex Trading Business For Beg...,https://www.udemy.com/introduction-to-forex-tr...,True,20,0,0,27,Beginner Level,1.500000,2017-04-23T16:19:01Z,Business Finance
894,1035638,Understanding Financial Statements,https://www.udemy.com/understanding-financial-...,True,25,0,0,10,All Levels,1.000000,2016-12-15T14:56:17Z,Business Finance
1100,1084454,CFA Level 2- Quantitative Methods,https://www.udemy.com/cfa-level-2-quantitative...,True,40,0,0,35,All Levels,5.500000,2017-07-02T14:29:35Z,Business Finance
1473,185526,MicroStation - Células,https://www.udemy.com/microstation-celulas/,True,20,0,0,9,Beginner Level,0.616667,2014-04-15T21:48:55Z,Graphic Design
2561,28295,Learn Web Designing & HTML5/CSS3 Essentials in...,https://www.udemy.com/build-beautiful-html5-we...,True,75,43285,525,24,All Levels,4.000000,2013-01-03T00:55:31Z,Web Development


In [54]:
data = data.drop_duplicates()

In [55]:
data.shape

(3672, 12)

In [56]:
data['course_title']

,course_title
0,Ultimate Investment Banking Course
1,Complete GST Course & Certification - Grow You...
2,Financial Modeling for Business Analysts and C...
3,Beginner to Pro - Financial Analysis in Excel ...
4,How To Maximize Your Profits Trading Options
...,...
3673,Learn jQuery from Scratch - Master of JavaScri...
3674,How To Design A WordPress Website With No Codi...
3675,Learn and Build using Polymer
3676,CSS Animations: Create Amazing Effects on Your...


In [57]:
data.columns

Index(['course_id', 'course_title', 'url', 'is_paid', 'price',
       'num_subscribers', 'num_reviews', 'num_lectures', 'level',
       'content_duration', 'published_timestamp', 'subject'],
      dtype='object')

### Popularity-based recommendation system

In [58]:
def popularity_based_recommendation(df,top_n=5):
    # Calculate popularity score for each course
    data['popularity_score'] = 0.6 * data['num_subscribers'] + 0.4 * data['num_reviews']

    # Sort courses by popularity score in descending order
    df_sorted = data.sort_values(by='popularity_score', ascending=False)

    # Return the recommended courses (course titles and popularity scores)
    recommended_courses = df_sorted[['course_title', 'popularity_score']].head(top_n)

    return recommended_courses

In [59]:
popularity_based_recommendation(data)

<ipython-input-58-d549335ae025>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['popularity_score'] = 0.6 * data['num_subscribers'] + 0.4 * data['num_reviews']


,course_title,popularity_score
2827,Learn HTML5 Programming From Scratch,164805.4
3032,Coding for Entrepreneurs Basic,96729.0
3230,The Web Developer Bootcamp,83928.4
3232,The Complete Web Developer Course 2.0,77672.0
2783,Build Your First Website in 1 Week with HTML5 ...,74544.2


### Content-Based Recommendation System

In [60]:
data['course_title'] = data['course_title'].apply(nfx.remove_stopwords)
data['course_title']  =data['course_title'].apply(nfx.remove_special_characters)

<ipython-input-60-7d1bc7695065>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['course_title'] = data['course_title'].apply(nfx.remove_stopwords)
<ipython-input-60-7d1bc7695065>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['course_title']  =data['course_title'].apply(nfx.remove_special_characters)


In [61]:
data.sample(5)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,popularity_score
3110,295843,Introduction Laravel 4,https://www.udemy.com/introduction-to-laravel-4/,False,0,9110,411,22,All Levels,2.5,2014-09-12T22:52:21Z,Web Development,5630.4
13,794151,Forex Trading Course Work Smarter Harder Prove...,https://www.udemy.com/the-complete-trading-cou...,True,195,4061,52,16,All Levels,2.0,2016-03-16T15:40:19Z,Business Finance,2457.4
720,253810,IAS 8Accounting Policies Changes Accounting Es...,https://www.udemy.com/ias-8-accounting-policie...,False,0,2122,30,15,All Levels,2.5,2014-10-03T12:29:41Z,Business Finance,1285.2
1000,619786,Profit Crashing Markets StepbyStep Guide,https://www.udemy.com/profit-during-crashing-m...,True,195,57,11,18,All Levels,1.0,2016-05-16T21:38:56Z,Business Finance,38.6
3563,830834,Quick learning jQuery web development,https://www.udemy.com/quick-learning-jquery-we...,True,150,3388,67,39,Intermediate Level,3.0,2016-04-25T17:04:36Z,Web Development,2059.6


In [62]:
data['title_subject']  =data['course_title'] +' '+data['subject']

<ipython-input-62-8bb55bbc92ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['title_subject']  =data['course_title'] +' '+data['subject']


In [63]:
cv = CountVectorizer(max_features=3000)
vectors = cv.fit_transform(data['title_subject']).toarray()

In [64]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [65]:
len(cv.get_feature_names_out())

3000

In [66]:
#cv.get_feature_names()

In [67]:
from sklearn.metrics.pairwise import cosine_similarity

In [68]:
similarity = cosine_similarity(vectors)

In [69]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(39, np.float64(0.7715167498104596)),
 (240, np.float64(0.6666666666666669)),
 (417, np.float64(0.6666666666666669)),
 (418, np.float64(0.6172133998483676)),
 (657, np.float64(0.6172133998483676))]

In [70]:
def recommend(course):
    # let's featch the index
    course_index = data[data['course_title']==course].index[0]
    distances = similarity[course_index]
    courses_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in courses_list:
        print(data.iloc[i[0]]['course_title'])

In [71]:
#recommend("know HTML Learn HTML Basics")

In [72]:
recommend("know HTML Learn HTML Basics")

WordPress Development Beginners
Wordpress Theme Development Beginners
Wordpress beginners Build Websites Fast Coding
Website Coding WordPress  Web Skills
Kids Coding  Beginners CSS


In [73]:
data.iloc[39]['course_title']

'Complete Investment Banking Course 2017'

In [74]:
#sorted(similarity[0],reverse=True)

In [75]:
import pickle

In [76]:
#pickle.dump(data.to_dict(),open('course_dict.pkl','wb'))
pickle.dump(data,open('course_dict.pkl','wb'))

In [77]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [78]:
# import tkinter as tk
# from tkinter import ttk, messagebox
# import pandas as pd

# # Assume 'data', 'similarity' are defined elsewhere

# # Define the popularity-based recommendation function
# def popularity_based_recommendation(df, top_n=5):
#     df['popularity_score'] = 0.6 * df['num_subscribers'] + 0.4 * df['num_reviews']
#     df_sorted = df.sort_values(by='popularity_score', ascending=False)
#     recommended_courses = df_sorted[['course_title', 'popularity_score']].head(top_n)
#     return recommended_courses

# # Define the recommend function
# def recommend(course):
#     try:
#         course_index = data[data['course_title'] == course].index[0]
#         distances = similarity[course_index]
#         courses_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
#         recommended_courses = [data.iloc[i[0]]['course_title'] for i in courses_list]
#         return recommended_courses
#     except IndexError:
#         messagebox.showerror("Error", f"Course '{course}' not found.")

# # Event handler for the "Recommend" button
# def recommend_button_click():
#     course_title = course_var.get()
#     recommended_courses = recommend(course_title)
#     if recommended_courses:
#         popularity_label.pack_forget()
#         result_label.config(text="Recommended Courses:\n" + '\n'.join(recommended_courses))

# # Create the main application window
# root = tk.Tk()
# root.title("Course Recommender")
# root.geometry("400x300")

# # Change font and color
# font_style = ("Arial", 12)
# label_color = "blue"
# heading_color="red"
# button_color = "green"
# result_label_color = "black"

# # Create and place GUI elements
# label = tk.Label(root, text="Select Course:", font=font_style, fg=label_color)
# label.pack(pady=10)

# course_titles = data['course_title'].tolist()
# course_var = tk.StringVar(value=course_titles[0])
# course_dropdown = ttk.Combobox(root, textvariable=course_var, values=course_titles, width=40, font=font_style)
# course_dropdown.pack(pady=5)

# popularity_recommendations = popularity_based_recommendation(data, top_n=5)
# popularity_label = tk.Label(root, text="Popularity-based Recommendations:\n" + popularity_recommendations.to_string(index=False),
#                              font=font_style, fg=label_color)
# popularity_label.pack()

# recommend_button = tk.Button(root, text="Recommend", command=recommend_button_click, width=20, font=font_style, fg=button_color)
# recommend_button.pack(pady=10)

# result_label = tk.Label(root, text="", wraplength=350, font=font_style, fg=result_label_color)
# result_label.pack()

# root.mainloop()


In [80]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assume 'data', 'similarity' are already defined

# Define the popularity-based recommendation function
def popularity_based_recommendation(df, top_n=5):
    df['popularity_score'] = 0.6 * df['num_subscribers'] + 0.4 * df['num_reviews']
    df_sorted = df.sort_values(by='popularity_score', ascending=False)
    recommended_courses = df_sorted[['course_title', 'popularity_score']].head(top_n)
    return recommended_courses

# Define the recommend function
def recommend(course):
    try:
        course_index = data[data['course_title'] == course].index[0]
        distances = similarity[course_index]
        courses_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
        recommended_courses = [data.iloc[i[0]]['course_title'] for i in courses_list]
        return recommended_courses
    except IndexError:
        return ["Error: Course not found."]

# Create dropdown menu for course selection
course_titles = data['course_title'].tolist()
course_dropdown = widgets.Dropdown(
    options=course_titles,
    value=course_titles[0],
    description='Course:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Button to trigger recommendation
recommend_button = widgets.Button(
    description='Recommend',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='30%')
)

# Output area
output = widgets.Output()

# Event handler when button is clicked
def recommend_button_click(b):
    with output:
        clear_output()
        selected_course = course_dropdown.value
        recommended_courses = recommend(selected_course)
        print("\nRecommended Courses:\n")
        for course in recommended_courses:
            print(f"- {course}")

# Display popularity-based recommendations first
popularity_recommendations = popularity_based_recommendation(data, top_n=5)

print("Popularity-Based Top 5 Courses:\n")
display(popularity_recommendations)

# Link the button click to the handler
recommend_button.on_click(recommend_button_click)

# Display widgets
display(course_dropdown)
display(recommend_button)
display(output)


Popularity-Based Top 5 Courses:



,course_title,popularity_score
2827,Learn HTML5 Programming Scratch,164805.4
3032,Coding Entrepreneurs Basic,96729.0
3230,Web Developer Bootcamp,83928.4
3232,Complete Web Developer Course 20,77672.0
2783,Build Website 1 Week HTML5 CSS3,74544.2


Dropdown(description='Course:', layout=Layout(width='70%'), options=('Ultimate Investment Banking Course', 'Co…

Button(button_style='success', description='Recommend', layout=Layout(width='30%'), style=ButtonStyle())

Output()